# Solar irradiance on sloping terrain

This module determines the terrain gradient, aspects and solar irradiance on a terrain specified by a 2-D array.

In [ ]:
import math
import numpy as np

In [1]:
def readarr(filename):
    """ Reads the digital elevation data stored in the form 
        of triplets (Easting, Northing, elevation) into
        a Python dictionary resembling a 2-D array,
        but more flexible in that it allows indices to
        store real false Easting/Northing (i.e. indicies
        not necessary starting from 0)."""
    with open(filename) as f:
        elevation = {}
        for line in f:
            x, y, z, *_ = line.split()   # discard remaining contents after z
            x, y, z = int(x), int(y), int(z)    # string --> int
            elevation[(x,y)] = z
        return elevation

In [2]:
elev = readarr('efyrnwy.dem')

Formulae for the EW gradient:

$$ \text{gradEW} = \frac{\text{elev}[E-\Delta, N] - \text{elev}[E + \Delta, N]}{2 \Delta} $$

the NS gradient:

$$ \text{gradNS} = \frac{\text{elev}[E, N+\Delta] - \text{elev}[E, N - \Delta]}{2 \Delta} $$

the overall slope angle:

$$ \theta = \tan^{-1} \left( \sqrt{\text{gradEW}^2 + \text{gradNS}^2} \right) $$

the aspect:

$$ \phi = 180 + \tan^{-1} \left( -\frac{\text{gradEW}}{\text{gradNS}} \right) $$

In [12]:
def gradasp(demfile, size):
    """ Takes a 2D DEM and for each cell determines the local 
    gradient and aspect using the method of Zevenbergen 
    and Thorne (1987). Prints out two files, each 
    containing the coordinates of the cell concerned 
    and either (i) its gradient or (ii) its aspect.
    
    Parameters
    ----------
    `demfile`   Name of DEM input file
    `size`      DEM cell size (metres)

    Variables
    ---------
    min_E      eastern-most Easting in DEM
    max_E      western-most Easting in DEM
    min_N      southern-most Northing in DEM
    min_N      northern-most Northing in DEM
    this_E     Easting of current cell
    this_N     Northing of current cell
    gradNS     Gradient of current cell measured North-South
    gradEW     Gradient of current cell measured East-West
    gradient   Gradient of current cell
    aspect     Aspect of current cell
    elevation  2D array storing the DEM
    rad2deg    Conversion factor - radians to degrees. """
    
    elev = {}
    with open(demfile) as f:
        first_line = True
        for line in f:
            E, N, z, *_ = line.split()   # discard remaining contents after z
            E, N, elev[(E,N)] = int(E), int(N), int(z)    # string --> int
            
            # Initialize minimum and maximum Easting and Northing 
            # of the DEM
            if first_line:
                min_E, max_E, min_N, max_N = E, E, N, N
                first_line = False
            
            if E < min_E: min_E = E   # Determine minimum and
            if E > max_E: max_E = E   # maximum Eastings and
            if N < min_N: min_N = N   # Northings (i.e., limits)
            if N > max_N: max_N = N   # of DEM
            
    with open("aspect.dem", "w") as f_asp, open("gradient.dem", "w") as f_grad:
        gradient = {}
        for this_E in range(min_E+size, max_E, size):
            for this_N in range(min_N+size, max_N, size):
                # Calculate Zevenbergen and Thorne's (1987) parameters
                gradEW = (elev[this_E-size,this_N] -
                            elev[this_E+size, this_N]) / (2*size)
                gradNS = (elev[this_E,this_N+size] -
                            elev[this_E, this_N-size]) / (2*size)
                
                # Calculate the gradient (degrees)
                gradient[(this_E,this_N)] = math.degrees(
                                            math.sqrt(gradNS**2 + gradEW**2))
                # Print this value out to a file
                f_grad.write("%6i %6i %5.2f\n" % (this_E, this_N, 
                                                gradient[this_E, this_N]))
                
                # Calculate the aspect (degrees)
                aspect = 180 + math.degrees(math.atan2(-gradEW, gradNS))
                # Print this value out to a file
                f_asp.write("%6i %6i %3i\n" % (this_E, this_N, aspect))


In [13]:
def demirrad(demfile, size, latitude, DOY, hour_angle):
   """ Takes a 2D DEM in the form Easting, Northing, elevation
      and determines the local gradient and aspect of
      each cell using the method of Zevenbergen and
      Thorne (1987). It then uses this information to
      calculate the direct, diffuse and total solar irradiance 
      on each cell given prescribed values of total exo-
      atmospheric solar irradiance, atmospheric transmittance
      and atmospheric pressue (at the altitude of the site and
      at sea level), and user-defined values of the latitude 
      of the site, the day of year and the solar hour angle.
   
   Variables:
   ----------
   E_0        Exo-atmospheric solar irradiance (W.m^{-2})
   tau        Atmospheric transmittance
   p_alt      Atmospheric pressure at average altitude (mbar)
   p_sea      Atmospheric pressure at sea level (mbar)
   rad2deg    Conversion factor - radians to degrees
   deg2rad    Conversion factor - degrees to radians
   latitude   Latitude of site (initially in degrees)
   hour_angle Solar hour angle at site (initially degrees)
   zenith     Solar zenith angle (radians)
   azimuth    Solar azimuth angle (radians)
   declinatn  Solar declination angle (degrees)
   air_mass   Atmospheric air mass
   zeta       Angle between sun and surface normal (radians)
   I_direct   Direct solar irradiance on surface (W.m^{-2})
   I_diffuse  Diffuse solar irradiance on surface (W.m^{-2})
   I_total    Total solar irradiance on surface (W.m^{-2})
   sky_view   Sky view factor
   min_E      eastern-most Easting in DEM (metres)
   max_E      western-most Easting in DEM (metres)
   min_N      southern-most Northing in DEM (metres)
   min_N      northern-most Northing in DEM (metres)
   this_E     Easting of current cell (metres)
   this_N     Northing of current cell (metres)
   size       DEM cell size (metres)
   grad_NS    N-Sg radient of current cell (degrees)
   grad_EW    E-W gradient of current cell (degrees)
   gradient   Gradient of current cell (degrees)
   aspect     Aspect of current cell (degrees)
   """

   # Initialize key variables
   E_0 = 1380.0
   tau = 0.7
   p_alt = 1000.0
   p_sea = 1013.0
   sin = math.sin
   cos = math.cos

   # Convert latitude and hour angle values provided
   # via command line from degrees to radians
   latitude = math.radians(latitude)
   hour_angle = math.radians(hour_angle)

   # Calculate solar declination angle (radians)
   declinatn = math.radians(-23.4) * cos(math.radians(360*(DOY+10)/365))

   # Calculate solar zenith angle (radians)
   if hour_angle < math.radians(90) or hour_angle > math.radians(-90):
      zenith = np.arccos(sin(latitude) * sin(declinatn) +
         cos(latitude)*cos(declinatn)*cos(hour_angle))
   else:
      zenith = np.arccos(sin(latitude) * sin(declinatn) -
         cos(latitude) * cos(declinatn) * cos(hour_angle))

   # Calculate solar azimuth angle (radians)
   if hour_angle > 0:
      azimuth = math.radians(180) + np.arccos(-1*(sin(declinatn) -
         cos(zenith)*sin(latitude)) / (cos(latitude)*sin(zenith)))
   else:
      azimuth = math.radians(180) - np.arccos(-1*(sin(declinatn) -
         cos(zenith)*sin(latitude)) / (cos(latitude)*sin(zenith)))
   
   # Calculate atmospheric air mass
   air_mass = (p_alt/p_sea) / cos(zenith)
   
   dem = {}
   with open(demfile) as f:
      first_line = True
      for line in f:
         E, N, z, *_ = line.split()   # discard remaining contents after z
         E, N, dem[(E,N)] = int(E), int(N), int(z)    # string --> int

         # Initialize minimum and maximum Easting & Northing of DEM
         if first_line:
               min_E, max_E, min_N, max_N = E, E, N, N
               first_line = False
         
         # Determine min. and max. Eastings and Northings of DEM
         if E < min_E: min_E = E   # Determine minimum and
         if E > max_E: max_E = E   # maximum Eastings and
         if N < min_N: min_N = N   # Northings (i.e., limits)
         if N > max_N: max_N = N   # of DEM

   # Pass across DEM by Easting
   with open("efyrnwy.dir", "w") as f_dir, \
         open("efyrnwy.dif", "w") as f_dif, \
         open("efyrnwy.tot", "w") as f_tot:
      gradient = {}
      for this_E in range(min_E+size, max_E, size):
         for this_N in range(min_N+size, max_N, size):
            # Calculate Zevenbergen and Thorne's (1987) parameters
            gradEW = (z[this_E-size, this_N] - z[this_E+size, this_N]) / (2*size)
            gradNS = (z[this_E, this_N+size] - z[this_E, this_N-size]) / (2*size)

            # Calculate the gradient (radians)
            gradient[(this_E,this_N)] = math.sqrt(gradNS**2 + gradEW**2)

            # Calculate the aspect (radians)
            aspect = math.pi + math.atan2(-gradEW, gradNS)

            # Calculate angle of incidence of solar radiation
            # for the inclined surface
            cos_zeta = cos(gradient) * cos(zenith) + \
                        sin(gradient) * sin(zenith) * cos(azimuth-aspect)
            
            # Calculate the direct solar irradiance
            I_direct = E_0 * (tau**air_mass) * cos_zeta;

            if I_direct < 0:
               I_direct = 0

            # Print out the Easting, Northing and direct solar
            # irradiance for this cell
            f_dir.write("%6d %6d %6.1f\n" % (this_E, this_N, I_direct))
      
            # Calculate the diffuse solar irradiance
            sky_view = 0.5 * (1 + cos(gradient));
            I_diffuse = sky_view * 0.3 * (1 - tau**air_mass) * E_0 * cos(zenith)
            if I_diffuse < 0:
               I_diffuse = 0

            # Print out the Easting, Northing and diffuse solar
            # irradiance for this cell
            f_dif.write("%6d %6d %6.1f\n" % (this_E, this_N, I_diffuse))

            # Calculate global solar irradiance on surface
            I_global = I_direct + I_diffuse;
            if I_global < 0:
               I_global = 0

            # Print out the Easting, Northing and diffuse solar
            # irradiance for this cell
            f_tot.write("%6d %6d %6.1f\n" % (this_E, this_N, I_global))
